In [ ]:
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from typing import Optional

# 메시지 플레이스 홀더가 있는 프롬프트 템플릿 정의
# 1. 입력 정리 프롬프트 - 날짜, 장소, 예산
class TravelPreference(BaseModel):
	"""여행 기본 정보"""
	place: str = Field(description="여행 장소(도시/지역)")
	date: str = Field(description="여행 날짜/기간 (예: 2026-02-01~2026-02-07 또는 2박 3일)")
	budget: Optional[int] = Field(default=None, description="예산(원 단위 정수), 없으면 None")


input_check_prompt = ChatPromptTemplate.from_messages([
	("system", 
	"너는 맞춤형 여행 일정 계획 어시스턴트야.\n"
	"사용자 입력과 chat_history에 입력된 대화를 보고 여행 기본 정보를 출력해.\n"
	"출력은 반드시 TravelPreference에 맞는 값으로 출력해\n"
	"예산은 원 단위 정수로, 모르면 null로 둬\n"
	"모든 내용은 한국어 기준으로 작성해!\n"
	),
	MessagesPlaceholder("chat_history"),
	("human", "{user_input}")
])

# 2. 여행 일정 생성 프롬프트
schedule_draft_prompt = ChatPromptTemplate.from_messages([
    ("system",
		"너는 맞춤형 여행 일정 계획 어시스턴트야.\n"
    "입력된 여행 기본 정보(place/date/budget)를 바탕으로 '일정 초안'을 만들어.\n"
    "반드시 한국어로.\n"
    "형식: Day 1 / Day 2 ... 각 Day는 아침/점심/저녁/야간(선택).\n"
    "예산이 있으면 대략 배분도 포함해."
    ),
    MessagesPlaceholder("chat_history"),
    ("human", "여행 기본 정보:\n{pref}\n\n사용자 요청:\n{user_input}")
])

# 3. 여행 현실적 스케줄 프롬프트
schedule_detail_prompt = ChatPromptTemplate.from_messages([
    ("system",
    "너는 맞춤형 여행 일정 계획 어시스턴트야.\n"
    "'일정 초안'을 더 실전적으로 다듬어.\n"
    "각 Day마다: 이동 팁 1개, 예약/대기 팁 1개, 대안 코스 1개를 추가해.\n"
    "마지막에 확인 질문 2개만 해."
    ),
    MessagesPlaceholder("chat_history"),
    ("human", "일정 초안:\n{draft}\n\n사용자 요청:\n{user_input}")
])

# LLM 모델 정의
def chat_model_function(model, temperature):
		if model == 'gpt-4.1-nano':
			return ChatOpenAI(
				model=model, 
				temperature=temperature,
				top_p=0.9,
				presence_penalty=0.3,
				frequency_penalty=0.3,
			)
		elif model == 'gemini-2.5-flash-lite':
			return ChatGoogleGenerativeAI(
				model=model, 
				temperature=temperature,
			)
		else:
			raise ValueError("지원하지 않는 모델: {model}")
	

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
parser = StrOutputParser()

llm = chat_model_function("gpt-4.1-nano", 0.7)

input_check_chain = input_check_prompt | llm.with_structured_output(TravelPreference)
schedule_draft_chain = schedule_draft_prompt | llm | parser
schedule_detail_chain = schedule_detail_prompt | llm | parser


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):
    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    history_messages.append(HumanMessage(content=message))
    
		# Step1: 스키마 객체로 입력 정리
    pref = input_check_chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })  # pref: TravelPreference

    # Step2: 일정 초안
    draft = schedule_draft_chain.invoke({
        "chat_history": history_messages,
        "pref": pref.model_dump(),   # dict로 넘기면 충분히 잘 읽음
        "user_input": message
    })

    # Step3: 일정 디테일
    final = schedule_detail_chain.invoke({
        "chat_history": history_messages,
        "draft": draft,
        "user_input": message
    })

    return final
# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
	fn=answer_invoke,         # 메시지 처리 함수
	title="맞춤형 여행 일정 계획 어시스턴트", # 채팅 인터페이스의 제목
  analytics_enabled=False
)

# Gradio 인터페이스 실행
demo.launch()


In [ ]:
demo.close()